### 다중 분류에서 이진 분류로(구현)

In [3]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        print(target_W)
        out = np.sum(target_W * h, axis=1)   # sum((2,3)*(2,3)) => (2,)  입력층의 결과랑 멤버곱 하고 열 방향 합
        # 즉 결과가 스칼라가 아니라 (2,)인 행만 존재하는 벡터

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1) # (1,)을 (1,1)로

        dtarget_W = dout * h # 1행1열인 dout을 1행 3열로 열을 복제해서 브로드 캐스팅(3,3,3) 해서 1행 3열인 h랑 곱해짐
        print(dtarget_W)
        self.embed.backward(dtarget_W)
        print(self.embed.grads[0])
        dh = dout * target_W
        return dh

In [4]:
a = np.arange(7).reshape(-1,1)
print(a)
W_out = np.repeat(a, 3, axis=1) # weight만들기 위해 열로 3개만큼 복제
print(W_out)

h = np.array([[3,4,5]])   # (1,3) 입력, 히든?

idx = np.array([1])
ed = EmbeddingDot(W_out)      # (1,3)(3,1) => (1,1)  sum([[3,4,5]]*[[1,1,1]],axis=1)=>[12]

out = ed.forward(h, idx)
print(out) # 결국 임베딩을 통해 say를 하나의 값으로 표현할 수 있다는 것이 핵심!! 
print(ed.cache) # 백업 결과 확인 

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]]
[[0 0 0]
 [1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]
 [5 5 5]
 [6 6 6]]
[[1 1 1]]
[12]
(array([[3, 4, 5]]), array([[1, 1, 1]]))


In [12]:
dout = np.array([3])
dh = ed.backward(dout)
print(ed.embed.grads)
print("dh = ",dh)

[[ 9 12 15]
 [18 21 24]]
[[ 0  0  0]
 [ 9 12 15]
 [18 21 24]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]
[array([[ 0,  0,  0],
       [ 9, 12, 15],
       [18, 21, 24],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0]])]
dh =  [[3 3 3]
 [6 6 6]]


In [10]:
# p.149
h = np.array([[3,4,5],
              [6,7,8]])   # (2,3) 맥락 벡터의 평균으로 만들어진 값
idx = np.array([1,2]) # (1,2)라는 정답 자리에 있는 애를 idx에 넣음
ed = EmbeddingDot(W_out)
ed.forward(h, idx)

[[1 1 1]
 [2 2 2]]


array([12, 42])

In [13]:
dout = np.array([3,4])
dh = ed.backward(dout)
print(dh)

[[ 9 12 15]
 [24 28 32]]
[[ 0  0  0]
 [ 9 12 15]
 [24 28 32]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]
[[3 3 3]
 [8 8 8]]


### 네거티브 샘플링의 샘플링 기법

In [25]:
import numpy as np

np.random.choice(10)

9

In [28]:
np.random.choice(10)

6

In [31]:
words = ['you','say','goodbye','I','hello','.']
np.random.choice(words) # 단어 중에 하나 랜덤 초이스. 그러나 확률을 넣지 않았으니 1/7 확률로 선택

'you'

In [33]:
np.random.choice(words, size=5) # 5개 복원 추출

array(['goodbye', '.', 'you', '.', 'you'], dtype='<U7')

In [36]:
np.random.choice(words, size=5, replace=False) # 5개 비복원

array(['say', 'you', 'hello', '.', 'goodbye'], dtype='<U7')

In [50]:
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p) # 확률에 근거하여 1개 추출
# 확률이 너무 낮은 애들은 선택이 안 될 가능성이 있기 때문에 약간 보정해야 함.
# 빈도수가 너무 낮은 애들은 네거티브 샘플링에 포함되어 학습될 수 없기 때문! => 확률을 끌어올림

'you'

In [59]:
p = [1000, 1010, 10]
p /= np.sum(p)
print(p)

[0.4950495 0.5       0.0049505]


In [60]:
p = [1000, 1010, 10]
new_p = np.power(p, 0.75) # 각각 0.75승을 하여 작은 수가 차지하는 비율을 조금 높임
new_p /= np.sum(new_p)
print(new_p)
# 위에서 0.5%였던 게 1.5%정도로 끌어올려짐.
# 이런 방법을 통해 빈도수가 낮은 애들도 선택될 수 있게 확률 올려줌

[0.4904092  0.49408269 0.0155081 ]


In [61]:
import collections # 딕셔너리로 카테고리 별로 카운팅 해줌

corpus = np.array([1,2,3,4,5,2,3,4,1,1,4,2,2,2])
counts = collections.Counter()
print(len(counts))
for word_id in corpus:
    counts[word_id] += 1
    
print(len(counts))
print(counts)
# 2는 5개, 1은 3개 ~.. 

0
5
Counter({2: 5, 1: 3, 4: 3, 3: 2, 5: 1})


In [138]:
GPU = False

class UnigramSampler: # 랜덤하게 선택하는 코드
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        print(counts)
        
        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        print(self.word_p)
        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
        print(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
                # 빈도가 높은 애들이 negative sample로 추출되게 하기 위해 확률에 기반하기 위해 p=p 를 넣는 게 포인트!!!
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [158]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3, 1, 1, 2])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

Counter({1: 4, 2: 3, 3: 2, 0: 1, 4: 1})
[1. 4. 3. 2. 1.]
[0.11376918 0.32178782 0.25933764 0.19133618 0.11376918]
[[2 3]
 [2 1]
 [2 3]]


In [159]:
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.c_[a,b]
print(c)

[[1 4]
 [2 5]
 [3 6]]


### 네거티브 샘플링 구현

In [ ]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh
